In [1]:
from action_language_interpreter.util import solve

In [2]:
time_starts_at_0_axiom = """
time(0).
"""

condition_axioms = """
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, F), not pos_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, -F), not neg_at(F, T).
conditionFails_at(C, T) :- fluent(F), time(T), condition_fluent(C, unk(F)), not unk_at(F, T).
"""

causes_axioms = """
% E causes F if C*
pos_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, F, C), occ_at(E, T), not conditionFails_at(C, T).
neg_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, -F, C), occ_at(E, T), not conditionFails_at(C, T).
unk_at(F, T+1) :- event(E), fluent(F), time(T), event_causes_if(E, unk(F), C), occ_at(E, T), not conditionFails_at(C, T).
"""

impossible_if_axiom = """
% E impossible_if C*
:- event(E), time(T), eventImpossible_if(E, C), occ_at(E, T), not conditionFails_at(C, T).
"""

if_axioms = """
% F if C*
pos_at(F, T) :- fluent(F), time(T), fluent_if(F, C), not conditionFails_at(C, T).
neg_at(F, T) :- fluent(F), time(T), fluent_if(-F, C), not conditionFails_at(C, T).
unk_at(F, T) :- fluent(F), time(T), fluent_if(unk(F), C), not conditionFails_at(C, T).
"""

inertia_axioms = """
pos_at(F, T+1) :- fluent(F), time(T), time(T+1), pos_at(F, T), not neg_at(F, T+1), not unk_at(F, T+1).
neg_at(F, T+1) :- fluent(F), time(T), time(T+1), neg_at(F, T), not pos_at(F, T+1), not unk_at(F, T+1).
unk_at(F, T+1) :- fluent(F), time(T), time(T+1), unk_at(F, T), not pos_at(F, T+1), not neg_at(F, T+1).
"""

consistency_axioms = """
:- fluent(F), time(T), pos_at(F, T), unk_at(F, T).
:- fluent(F), time(T), neg_at(F, T), unk_at(F, T).
"""

completion_g1_axiom = """
% g1
pos_at(F, 0) :- fluent(F), init(F).
neg_at(F, 0) :- fluent(F), init(-F).
"""

completion_g2_axiom = """
% g2
pos_at(F, 0) :- fluent(F), forced(F), default(F), not init(-F).
1 { pos_at(F, 0); neg_at(F, 0) } 1 :- fluent(F), forced(F), not default(F), not init(F), not init(-F).
"""

completion_g3_axiom = """
% g3
neg_at(F, 0) :- fluent(F), default(F), not pos_at(F, 0).
unk_at(F, 0) :- fluent(F), not default(F), not pos_at(F, 0), not neg_at(F, 0).
"""

axioms = (time_starts_at_0_axiom, condition_axioms, causes_axioms, impossible_if_axiom, if_axioms, inertia_axioms,
          consistency_axioms, completion_g1_axiom, completion_g2_axiom, completion_g3_axiom)

In [3]:
force_all_fluents = """

forced(F) :- fluent(F).

"""

In [4]:
force_all_non_default = """

forced(F) :- fluent(F), not default(F).

"""

In [5]:
reason_by_case_for_all_fluents = """

split_at(F, T) :- fluent(F), occ_at(_, T).

"""

In [6]:
steps = """
%% Steps
time(0..1).
"""
events = """
%% Events
event(d). % John goes on a first date with Mary.
"""
fluents = """
%% Fluents
fluent(m). % John is married.
fluent(ab). % John is abnormal.
"""
knowledge_base = """
%% Knowledge Base
% Action Description
eventImpossible_if(d, 1).
condition_fluent(1, m).
condition_fluent(1, -ab).
"""
defaults = """
% Default fluents
default(ab).
"""
query = """
%% Query
query(m).
"""
action_sequence = """
action_sequence(d, 0).
"""
instance = (steps, events, fluents, knowledge_base, defaults, query, action_sequence)

In [7]:
initial_knowledge = """
% empty
"""

In [8]:
every_action = """
occ_at(E, T) :- event(E), time(T), action_sequence(E, T).
"""

In [9]:
solve([*axioms,
       *instance,
       initial_knowledge,
       every_action,
       force_all_non_default,
       reason_by_case_for_all_fluents], clingo_args=("--models", "0", "--enum-mode", "cautious"));

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) forced(m) query(m) time(0) time(1) action_sequence(d,0) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) neg_at(m,0) neg_at(m,1) occ_at(d,0) split_at(ab,0) split_at(m,0) }.
SAT 1 


<block>:3:28-35: info: atom does not occur in any rule head:
  init(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:3:55-63: info: atom does not occur in any rule head:
  init((-F))

<block>:4:28-36: info: atom does not occur in any rule head:
  init((-F))

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:4:81-88: info: atom does not occur in any rule head:
  init(F)

<block>:4:94-102: info: atom does not occur in any rule head:
  init((-F))

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in any rule head:
  event_causes_if(E,unk(F),C)



In [10]:
# Answer set from above as facts.
R = """

default(ab).
event(d).
fluent(ab).
fluent(m).
forced(m).
query(m).
time(0).
time(1).
conditionFails_at(1,0).
conditionFails_at(1,1).
condition_fluent(1,m).
condition_fluent(1,-ab).
eventImpossible_if(d,1).
neg_at(ab,0).
neg_at(ab,1).
neg_at(m,0).
neg_at(m,1).
occ_at(d,0).
split_at(ab,0).
split_at(m,0).

"""

In [11]:
make_conservative_assumptions = """

init(F) :- fluent(F), forced(F), pos_at(F, 0).
init(-F) :- fluent(F), forced(F), neg_at(F, 0).

#show init/1.

"""

In [12]:
solve([R, make_conservative_assumptions]);

Answer  1: { init(-m) }.
SAT 1 


<block>:3:34-46: info: atom does not occur in any rule head:
  pos_at(F,0)



In [13]:
extended_initial_knowledge = """

init(-m).

"""

In [14]:
enumerate_all_forcings_and_actions = """

{ occ_at(E, T) : event(E), time(T), action_sequence(E, T) }.
{ forced(F) : fluent(F), not init(F), not init(-F) }.

#show occ_at/2.
#show forced/1.

"""

In [15]:
# FUTURE: async -> so that not all AS have to be enumerated
solve([steps,
       events,
       fluents,
       initial_knowledge,
       extended_initial_knowledge,
       action_sequence,
       enumerate_all_forcings_and_actions]);

Answer  1: {  }.
Answer  2: { occ_at(d,0) }.
Answer  3: { forced(ab) }.
Answer  4: { forced(ab) occ_at(d,0) }.
SAT 4 


In [16]:
must_finally_answer_query = """

:- query(Q), time(T), T = #max { T : occ_at(_, T) }, not pos_at(Q, T+1), not neg_at(Q, T+1).

"""

In [17]:
solve([*axioms,
       *instance,
       initial_knowledge,
       extended_initial_knowledge,
       #enumeration,
       must_finally_answer_query
 ]);

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) init(-m) query(m) time(0) time(1) action_sequence(d,0) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) neg_at(m,0) neg_at(m,1) }.
SAT 1 


<block>:3:34-35: info: global variable in tuple of aggregate element:
  T

<block>:3:28-37: info: atom does not occur in any rule head:
  forced(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:38-50: info: atom does not occur in any rule head:
  occ_at(#Anon0,T)

<block>:3:49-61: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:3:75-87: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:4:50-59: info: atom does not occur in any rule head:
  forced(F)

<block>:4:76-88: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in 

In [18]:
A = """
default(ab).
event(d).
fluent(ab).
fluent(m).
init(-m).
query(m).
time(0).
time(1).
action_sequence(d,0).
conditionFails_at(1,0).
conditionFails_at(1,1).
condition_fluent(1,m).
condition_fluent(1,-ab).
eventImpossible_if(d,1).
neg_at(ab,0).
neg_at(ab,1).
neg_at(m,0).
neg_at(m,1).
"""

In [19]:
extract_ground_knowledge = """

groundKnowledge(F) :- fluent(F), pos_at(F, 0), not init(F).
groundKnowledge(-F) :- fluent(F), neg_at(F, 0), not init(-F).

#show groundKnowledge/1.

"""

In [20]:
solve([A, extract_ground_knowledge]);

Answer  1: { groundKnowledge(-ab) }.
SAT 1 


<block>:3:34-46: info: atom does not occur in any rule head:
  pos_at(F,0)



In [21]:
X = """
init(-ab).
"""

In [22]:
solve([*axioms,
       *instance,
       X]);

Answer  1: { default(ab) event(d) fluent(ab) fluent(m) init(-ab) query(m) time(0) time(1) action_sequence(d,0) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) unk_at(m,0) unk_at(m,1) }.
SAT 1 


<block>:3:28-37: info: atom does not occur in any rule head:
  forced(F)

<block>:3:37-52: info: atom does not occur in any rule head:
  fluent_if(F,C)

<block>:3:49-61: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:3:49-73: info: atom does not occur in any rule head:
  event_causes_if(E,F,C)

<block>:3:75-87: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:4:37-53: info: atom does not occur in any rule head:
  fluent_if((-F),C)

<block>:4:49-74: info: atom does not occur in any rule head:
  event_causes_if(E,(-F),C)

<block>:4:50-59: info: atom does not occur in any rule head:
  forced(F)

<block>:4:76-88: info: atom does not occur in any rule head:
  occ_at(E,T)

<block>:5:37-57: info: atom does not occur in any rule head:
  fluent_if(unk(F),C)

<block>:5:49-78: info: atom does not occur in any rule head:
  event_causes_if(E,unk(F),C)

<block>:5:80-92: info: atom does not occur in any rule head:
  occ_at(E,T)



In [23]:
B = """
default(ab).
event(d).
fluent(ab).
fluent(m).
init(-ab).
query(m).
time(0).
time(1).
action_sequence(d,0).
conditionFails_at(1,0).
conditionFails_at(1,1).
condition_fluent(1,m).
condition_fluent(1,-ab).
eventImpossible_if(d,1).
neg_at(ab,0).
neg_at(ab,1).
unk_at(m,0).
unk_at(m,1).
"""

In [24]:
check_relevancy = """

relevant :- query(Q), not pos_at(Q, 0).
relevant :- query(Q), not neg_at(Q, 0).

:- not relevant.

"""

In [25]:
solve([B,
       check_relevancy,
       ]);

Answer  1: { relevant default(ab) event(d) fluent(ab) fluent(m) init(-ab) query(m) time(0) time(1) action_sequence(d,0) conditionFails_at(1,0) conditionFails_at(1,1) condition_fluent(1,m) condition_fluent(1,-ab) eventImpossible_if(d,1) neg_at(ab,0) neg_at(ab,1) unk_at(m,0) unk_at(m,1) }.
SAT 1 


<block>:3:27-39: info: atom does not occur in any rule head:
  pos_at(Q,0)



If relevant then return A.

In [26]:
calculate_relevance_score = """

relevanceScore(Forced + Split) :- Forced = #count { F : fluent(F), forced(F) }, Split = #count {F,T : fluent(F), time(T), split_at(F, T) }.

#show relevanceScore/1.

"""

In [27]:
solve([A,
       calculate_relevance_score]);

Answer  1: { relevanceScore(0) }.
SAT 1 


<block>:3:68-77: info: atom does not occur in any rule head:
  forced(F)

<block>:3:123-137: info: atom does not occur in any rule head:
  split_at(F,T)

